Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import time
localtime = time.asctime( time.localtime(time.time()) )
print (localtime)

Tue Jul  5 22:38:10 2016


First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic Regression

In [ ]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  beta = tf.constant(0.5)
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + tf.nn.l2_loss(weights) * beta
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Run the logistics regression with L2 loss (Full data set)


In [ ]:
num_steps = 4001
loss_list = []
with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    loss_list.append(l)
    if (step % 400 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.2f%%' % accuracy(
        predictions, train_labels[:train_subset]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.2f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print ('Test accuracy: %.2f%%' % accuracy(test_prediction.eval(), test_labels)) 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_list)
plt.show()

## Logistic Regression with stochastic descent algo

In [ ]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  beta = tf.constant(0.5)
    
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + tf.nn.l2_loss(weights) * beta
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Run the model

In [ ]:
num_steps = 3001
loss_list = []
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # Generate a minibatch.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_labels = train_labels[offset:(offset + batch_size), :]
    
    
    #Using permutation to randomnilized the data
    permutation = np.random.permutation(train_labels.shape[0])
    idx = permutation[:batch_size]
    #Generate a minibatch
    batch_data = train_dataset[idx, :]
    batch_labels = train_labels[idx,:]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    loss_list.append(l)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_list)
plt.show()

## Neural Network

### One hidden Level

In [ ]:
batch_size = 128
n_hidden = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  tf_hidden_dataset = tf.placeholder(tf.float32, shape = (batch_size, n_hidden))
    
  

  # Variables.
  weights_0 = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_0 = tf.Variable(tf.zeros([n_hidden]))
    
  weights_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_1 = tf.Variable(tf.zeros([num_labels]))
  beta = tf.constant(0.1)
  
  # Training computation.
  hidden_level = tf.matmul(tf_train_dataset, weights_0) + biases_0
  tf_hidden_dataset = tf.nn.relu(hidden_level)
  logits = tf.matmul(tf_hidden_dataset, weights_1) + biases_1
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + (tf.nn.l2_loss(weights_0) + tf.nn.l2_loss(weights_1)) * beta
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  

  def multilayer_perceptron(x, w_0, b_0,w_1, b_1):
        l1 = tf.nn.relu(tf.matmul(x, w_0) + b_0)
        pred = tf.matmul(l1, w_1) + b_1
        return pred
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights_0, biases_0, weights_1, biases_1))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights_0, biases_0, weights_1, biases_1))
  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_0) + biases_0), weights_1) + biases_1)
  #test_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_0) + biases_0), weights_1) + biases_1))
  print (r'Run the following model!')

In [ ]:
num_steps = 2001
rows = train_labels.shape[0]
loss_list = []
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    permutation = np.random.permutation(train_labels.shape[0])
    idx = permutation[:batch_size]
    #Generate a minibatch
    batch_data = train_dataset[idx, :]
    batch_labels = train_labels[idx,:]
    # Generate a minibatch.
    #batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    loss_list.append(l)
    if (step % 200 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.2f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_list[400::])
plt.show()

### Two hidden level

In [49]:
batch_size = 128
n_hidden = [1024,528]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  tf_hidden_dataset_1 = tf.placeholder(tf.float32, shape = (batch_size, n_hidden[0]))
  tf_hidden_dataset_2 = tf.placeholder(tf.float32, shape = (batch_size, n_hidden[1]))
    
  

  # Variables.
  weights_0 = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hidden[0]]))
  biases_0 = tf.Variable(tf.zeros([n_hidden[0]]))
  
  weights_1 = tf.Variable(
    tf.truncated_normal([n_hidden[0], n_hidden[1]]))
  biases_1 = tf.Variable(tf.zeros([n_hidden[1]])) 
    
  weights_2 = tf.Variable(tf.truncated_normal([n_hidden[1], num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  beta = tf.constant(0.1)
  # Training computation.
  hidden_level_1 = tf.matmul(tf_train_dataset, weights_0) + biases_0
  tf_hidden_dataset_1 = tf.nn.relu(hidden_level_1)
  hidden_level_2 = tf.matmul(tf_hidden_dataset_1, weights_1) + biases_1
  tf_hidden_dataset_2 = tf.nn.relu(hidden_level_2)
    
  logits = tf.matmul(tf_hidden_dataset_2, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + (tf.nn.l2_loss(weights_0) + tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(
    weights_2) + tf.nn.l2_loss(biases_0) + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(biases_2)) * beta
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.002).minimize(loss)
  

  def multilayer_perceptron(x, w_0, b_0,w_1, b_1, w_2, b_2):
        l1 = tf.nn.relu(tf.matmul(x, w_0) + b_0)
        l2 = tf.nn.relu(tf.matmul(l1,w_1) + b_1)
        pred = tf.matmul(l2, w_2) + b_2
        return pred
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights_0, biases_0, weights_1, biases_1, weights_2, biases_2))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights_0, biases_0, weights_1, biases_1, weights_2, biases_2))
  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_0) + biases_0), weights_1) + biases_1)
  #test_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_0) + biases_0), weights_1) + biases_1))
  print (r'Run the following model!')

Run the following model!


In [50]:
num_steps = 5001
rows = train_labels.shape[0]
loss_list = []
train_accuracy = []
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    permutation = np.random.permutation(train_labels.shape[0])
    idx = permutation[:batch_size]
    #Generate a minibatch
    batch_data = train_dataset[idx, :]
    batch_labels = train_labels[idx,:]
    # Generate a minibatch.
    #batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    loss_list.append(l)
    train_accuracy.append(accuracy(predictions, batch_labels))
    if (step % 200 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.2f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 57418.320312
Minibatch accuracy: 10.16%
Validation accuracy: 14.81%
Minibatch loss at step 200: 48436.464844
Minibatch accuracy: 70.31%
Validation accuracy: 74.00%
Minibatch loss at step 400: 44518.035156
Minibatch accuracy: 84.38%
Validation accuracy: 77.71%
Minibatch loss at step 600: 41088.519531
Minibatch accuracy: 75.78%
Validation accuracy: 77.91%
Minibatch loss at step 800: 37817.433594
Minibatch accuracy: 85.94%
Validation accuracy: 78.60%
Minibatch loss at step 1000: 34915.656250
Minibatch accuracy: 86.72%
Validation accuracy: 78.64%
Minibatch loss at step 1200: 32216.841797
Minibatch accuracy: 87.50%
Validation accuracy: 78.77%
Minibatch loss at step 1400: 29728.058594
Minibatch accuracy: 88.28%
Validation accuracy: 78.81%
Minibatch loss at step 1600: 27435.839844
Minibatch accuracy: 88.28%
Validation accuracy: 79.13%
Minibatch loss at step 1800: 25311.039062
Minibatch accuracy: 89.84%
Validation accuracy: 79.20%
Minibatch loss at step 20

In [51]:
import matplotlib.pyplot as plt
plt.subplot(2,1,1)
plt.plot(loss_list)
plt.subplot(2,1,2)
plt.plot(train_accuracy)
plt.show()

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
